In [2]:
from signal import signal, SIGPIPE, SIG_DFL
signal(SIGPIPE, SIG_DFL)

export CLICOLOR='1'
export CLUTTER_IM_MODULE='xim'
export CONDA_DEFAULT_ENV='Lab1_B'
export CONDA_PREFIX='/home/fecke/anaconda3/envs/Lab1_B'
export CONDA_PROMPT_MODIFIER='(Lab1_B) '
export CONDA_SHLVL='1'
export DBUS_SESSION_BUS_ADDRESS='unix:path=/run/user/1000/bus'
export DESKTOP_SESSION='ubuntu'
export DISPLAY=':0'
export GDMSESSION='ubuntu'
export GIO_LAUNCHED_DESKTOP_FILE='/home/fecke/.local/share/applications/jetbrains-pycharm.desktop'
export GIO_LAUNCHED_DESKTOP_FILE_PID='31692'
export GIT_PAGER='cat'
export GJS_DEBUG_OUTPUT='stderr'
export GJS_DEBUG_TOPICS='JS ERROR;JS LOG'
export GNOME_DESKTOP_SESSION_ID='this-is-deprecated'
export GNOME_SHELL_SESSION_MODE='ubuntu'
export GPG_AGENT_INFO='/run/user/1000/gnupg/S.gpg-agent:0:1'
export GTK_IM_MODULE='ibus'
export GTK_MODULES='gail:atk-bridge'
export HOME='/home/fecke'
export IM_CONFIG_PHASE='2'
export JPY_PARENT_PID='6228'
export KERNEL_LAUNCH_TIMEOUT='40'
export LANG='it_IT.UTF-8'
export LOGNAME='fecke'
export MPLBACKEND='module:

<Handlers.SIG_IGN: 1>

# Blast based method - workflow
This exercise is essential to learn how to build an Hidden Markov Model and it is necessary to follow the flowchart in order to get done the project for LAB_1 - Capriotti

*We want to build a blast-based method to predict the presence of BPTI/Kunitz domain in proteins available in SwissProt using the human proteins as a reference*
1. Select all proteins in SwissProt with BPTI/Kunitz domain
- Separate human from non human proteins. Une the non human proteins as a positive in the testing set
- Generate a random set of negative of the same size of the positive set
- Remove both positives and negatives from SwissProt and perform the prediction based on the result of the blast search

**The method should be based on the E-value.**

_For this exercise I will use blast-legacy2.2.26 from bioconda_

We will have:
set|Training|Testing
----|--------|-------
negative| HumanNOTPF_training.fasta| HumanNOTPF_testing.fasta
positive| NOTHumanPF00014|NOTHumanPF00014

## First step - retrieving data
- In order to select our dataset we will perform a UniProt search firstly by searching for those human protein that present the Kunitz domain (_Pfam: PF00014_) with the following:

- then we will find all those proteins which are not from human organism but present the Kunitz domain:

in order to find how many sequences are in each file we'll do:

In [3]:
!grep '>' ../databases/HumanPF00014.fasta |wc -l

18


In [4]:
!grep '>' ../databases/NOTHumanPF00014.fasta |wc -l

341


- In order to make this two files into database format we can use

In [5]:
!makeblastdb -in  ../databases/HumanPF00014.fasta -dbtype prot
!makeblastdb -in ../databases/NOTHumanPF00014.fasta -dbtype prot




Building a new DB, current time: 04/19/2020 00:11:37
New DB name:   /home/fecke/PycharmProjects/Lab1_B/databases/HumanPF00014.fasta
New DB title:  ../databases/HumanPF00014.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/fecke/PycharmProjects/Lab1_B/databases/HumanPF00014.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 18 sequences in 0.00647497 seconds.


Building a new DB, current time: 04/19/2020 00:11:37
New DB name:   /home/fecke/PycharmProjects/Lab1_B/databases/NOTHumanPF00014.fasta
New DB title:  ../databases/NOTHumanPF00014.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/fecke/PycharmProjects/Lab1_B/databases/NOTHumanPF00014.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 341 sequences in 0.0137641 seconds.


- This two databases are not so large, therefore result could be not so accurate, we would need a bigger dataset. We could search for all SwissProt protein that belongs to humans and do not present the Kunitz domain with the query:

In [6]:
!makeblastdb -in ../databases/HumanNOTPF00014.fasta -dbtype prot




Building a new DB, current time: 04/19/2020 00:12:32
New DB name:   /home/fecke/PycharmProjects/Lab1_B/databases/HumanNOTPF00014.fasta
New DB title:  ../databases/HumanNOTPF00014.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/fecke/PycharmProjects/Lab1_B/databases/HumanNOTPF00014.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 20347 sequences in 0.753836 seconds.


Let's check how many sequences are there:

In [7]:
!grep '>' ../databases/HumanNOTPF00014.fasta | wc -l

20347


## Second step - creating testing and training sets

my negative database will be --> ```HumanNOTPF00014.fasta```

I need to split my database in half:
- Firstly I need to find what is the sequence that splits it in halves. We know that sequences in ```HumanNOTPF00014``` are 20347, so we can split at 10170 sequence, more less at 1000. Let's see:

In [8]:
# Trova le prime 10000 sequenze che iniziano con ">" | mostra solo l'ultima
!grep ">" ../databases/HumanNOTPF00014.fasta -m 10170|tail -1
# Trova le prime 10001 seq che iniziano con ">" e printa il numero della linea + output | mostra solo l'ultima
!grep ">" ../databases/HumanNOTPF00014.fasta -m 10171 -n|tail -1

>sp|Q96HC4|PDLI5_HUMAN PDZ and LIM domain protein 5 OS=Homo sapiens OX=9606 GN=PDLIM5 PE=1 SV=5
110690:>sp|Q7Z417|NUFP2_HUMAN Nuclear fragile X mental retardation-interacting protein 2 OS=Homo sapiens OX=9606 GN=NUFIP2 PE=1 SV=1


on line ```110690``` we can retrieve the 10171st sequence. We can split at line ```110690 - 1 = 110689```. We will store these two parts of our HumanNOTPF00014.fasta in two different files. The first 10170 sequences will be used as our *Training set* while the other half will be used as a *Testing set*

In [9]:
# from the top until line 110689
!head -n 110689 ../databases/HumanNOTPF00014.fasta  > ../databases/HumanNOTPF_training.fasta
# from the bottom until it reaches line 110690
!tail -n +110690 ../databases/HumanNOTPF00014.fasta > ../databases/HumanNOTPF_testing.fasta

In [10]:
!grep '>' ../databases/HumanNOTPF_training.fasta -n | wc
!grep '>' ../databases/HumanNOTPF_testing.fasta | wc

  10170  111144 1083445
  10177  111709 1019599


## Consistency test
as a test we will perform blast of Human protein against itself. To have the results in tabular form we will use ```-outfmt 6```

In [7]:
!blastp -query ../databases/HumanPF00014.fasta -db ../databases/HumanPF00014.fasta -outfmt 6 -out HumanPF00014.bl8
!grep 'sp' HumanPF00014.bl8 | wc

    835   10020   73292


The 11th column corresponds to the e-value. We want to sort the ```HumanPF00014.bl8``` by considering the e-value column, from the worst (highest) to the best (lowest)

In [ ]:
#the first part is necessary in order to let the sort get the scientific notation
!LC_NUMERIC="en_US.UTF-8" sort -k11 -gr HumanPF00014.bl8 | head

sp|Q9BQY6|WFDC6_HUMAN   sp|Q02388|CO7A1_HUMAN   48.000  25      11      2       12      35      2315    2338    10.0    16.9
sp|Q8IUA0|WFDC8_HUMAN   sp|P12111|CO6A3_HUMAN   66.667  6       2       0       153     158     2395    2400    9.8     18.1
sp|P12111|CO6A3_HUMAN   sp|Q8TEU8|WFKN2_HUMAN   28.788  66      47      0       2897    2962    266     331     9.8     21.9
sp|P02760|AMBP_HUMAN    sp|Q02388|CO7A1_HUMAN   45.833  24      11      1       7       28      207     230     9.6     18.9
sp|Q8IUA0|WFDC8_HUMAN   sp|Q9BQY6|WFDC6_HUMAN   66.667  9       3       0       174     182     53      61      9.5     17.7
sp|P02760|AMBP_HUMAN    sp|Q9BQY6|WFDC6_HUMAN   34.783  23      15      0       93      115     91      113     9.4     18.1
sp|O95428|PPN_HUMAN     sp|Q2UY09|COSA1_HUMAN   66.667  12      4       0       889     900     303     314     9.3     20.8
sp|O95428|PPN_HUMAN     sp|O43291|SPIT2_HUMAN   33.333  27      15      1       151     174     60      86      8.9     20.4


If we want to retrieve all the proteins that were blasted with theirselves we can do:

In [21]:
!awk '{if ($1==$2) print $0}' HumanPF00014.bl8 | less

sp|Q06481|APLP2_HUMAN   sp|Q06481|APLP2_HUMAN   100.000 763     0       0       1       763     1       763     0.0     1590
sp|P05067|A4_HUMAN      sp|P05067|A4_HUMAN      100.000 770     0       0       1       770     1       770     0.0     1610
sp|Q02388|CO7A1_HUMAN   sp|Q02388|CO7A1_HUMAN   100.000 2944    0       0       1       2944    1       2944    0.0     5494
sp|P12111|CO6A3_HUMAN   sp|P12111|CO6A3_HUMAN   100.000 3177    0       0       1       3177    1       3177    0.0     6455
sp|P02760|AMBP_HUMAN    sp|P02760|AMBP_HUMAN    100.000 352     0       0       1       352     1       352     0.0     734
sp|O95925|EPPI_HUMAN    sp|O95925|EPPI_HUMAN    100.000 133     0       0       1       133     1       133     8.42e-100       274
sp|P10646|TFPI1_HUMAN   sp|P10646|TFPI1_HUMAN   100.000 304     0       0       1       304     1       304     0.0     630
sp|O43278|SPIT1_HUMAN   sp|O43278|SPIT1_HUMAN   100.000 529     0       0       1       529     1       529     0.0     

We can check if they are 18 by doing:

In [20]:
!awk '{if ($1==$2) print $0}' HumanPF00014.bl8 | wc -l

18


## Training
### Run on negatives
We want to get the negatives, let's blast it.

In [17]:
!blastp -query ../databases/HumanNOTPF_training.fasta -db ../databases/HumanPF00014.fasta -outfmt 6 -out HumanNOTPF00014_train.bl8

In [19]:
!wc -l HumanNOTPF00014_train.bl8

58954 HumanNOTPF00014_train.bl8


At this point I want to know the best hits between HumanNOTPF and HumanPF. So we sort:

In [5]:
!LC_NUMERIC="en_US.UTF-8" sort -k11 -g HumanNOTPF00014_train.bl8 | head

sp|A6NMZ7|CO6A6_HUMAN	sp|P12111|CO6A3_HUMAN	32.023	2239	1310	43	27	2138	639	2792	0.0	1013
sp|A8TX70|CO6A5_HUMAN	sp|P12111|CO6A3_HUMAN	31.051	2235	1320	51	30	2130	639	2786	0.0	927
sp|A6NMZ7|CO6A6_HUMAN	sp|P12111|CO6A3_HUMAN	28.559	1194	795	25	26	1181	38	1211	5.81e-140	480
sp|P51693|APLP1_HUMAN	sp|Q06481|APLP2_HUMAN	35.109	732	350	19	37	648	34	760	3.59e-126	383
sp|A8TX70|CO6A5_HUMAN	sp|P12111|CO6A3_HUMAN	27.288	1180	802	28	29	1172	38	1197	2.94e-121	421
sp|A6NMZ7|CO6A6_HUMAN	sp|P12111|CO6A3_HUMAN	28.182	1100	704	31	226	1272	36	1102	2.43e-110	385
sp|Q9UKP5|ATS6_HUMAN	sp|O95428|PPN_HUMAN	40.202	495	274	11	561	1045	29	511	2.62e-108	359
sp|Q8TE60|ATS18_HUMAN	sp|O95428|PPN_HUMAN	36.842	532	306	11	592	1115	29	538	1.04e-102	345
sp|Q9H324|ATS10_HUMAN	sp|O95428|PPN_HUMAN	37.996	529	306	12	550	1064	29	549	6.93e-99	332
sp|Q8TE57|ATS16_HUMAN	sp|O95428|PPN_HUMAN	36.684	567	320	16	589	1133	29	578	5.59e-96	326


The first two lines expresses all those sequences that have e-value 0 and we can assume that there are two protein sequences, one that present PF00014 domain and another one which does not, and that means that these sequences have similar domains.
```sp|A6NMZ7|CO6A6_HUMAN	sp|P12111|CO6A3_HUMAN	32.023	2239	1310	43	27	2138	639	2792	0.0	1013```

```sp|A8TX70|CO6A5_HUMAN	sp|P12111|CO6A3_HUMAN	31.051	2235	1320	51	30	2130	639	2786	0.0	927```
### Run on positives
We need to blast on the testing set. Testing set contains ```NOTHumanBPTI``` with ```HumanPF.fasta```.

In [6]:
!blastp -query ../databases/NOTHumanPF00014.fasta -db ../databases/HumanPF00014.fasta -outfmt 6 -out NOTHumanPF00014.bl8 -evalue 1000

In [3]:
!LC_NUMERIC=C sort -g -k11 NOTHumanPF00014.bl8 | head

sp|P00978|AMBP_BOVIN	sp|P02760|AMBP_HUMAN	77.273	352	80	0	1	352	1	352	0.0	561
sp|P04366|AMBP_PIG	sp|P02760|AMBP_HUMAN	80.655	336	65	0	2	337	17	352	0.0	581
sp|P08592|A4_RAT	sp|P05067|A4_HUMAN	96.883	770	24	0	1	770	1	770	0.0	1557
sp|P08592|A4_RAT	sp|Q06481|APLP2_HUMAN	49.873	790	320	17	3	769	26	762	0.0	716
sp|P0C5J5|WFKN1_RAT	sp|Q8TEU8|WFKN2_HUMAN	56.691	538	209	8	31	550	41	572	0.0	607
sp|P0C5J5|WFKN1_RAT	sp|Q96NZ8|WFKN1_HUMAN	87.992	533	63	1	20	552	17	548	0.0	939
sp|P12023|A4_MOUSE	sp|P05067|A4_HUMAN	96.623	770	26	0	1	770	1	770	0.0	1551
sp|P12023|A4_MOUSE	sp|Q06481|APLP2_HUMAN	50.253	790	317	17	3	769	26	762	0.0	721
sp|P15943|APLP2_RAT	sp|Q06481|APLP2_HUMAN	92.318	742	51	2	26	765	26	763	0.0	1340
sp|P15989|CO6A3_CHICK	sp|P12111|CO6A3_HUMAN	65.153	2884	977	11	1	2873	1	2867	0.0	3726


Results are in two files, I can generate a HMMlogo by going on https://skylign.org/

$Accuracy = \frac{TP+TN}{everything}$
 

$MCC =\frac{TP∗TF+FP∗FN}{(TP+FP)∗(TP∗FN)∗(TN+FP)∗(TN+FN)}$